# Pixie: pixel clustering notebook

In [ ]:
# import required packages
import json
import os
from datetime import datetime as dt

import matplotlib.pyplot as plt
from alpineer import io_utils, load_utils
from matplotlib import rc_file_defaults

from ark.phenotyping import (pixel_cluster_utils, pixel_meta_clustering,
                             pixel_som_clustering, pixie_preprocessing)
from ark.utils import data_utils, example_dataset, plot_utils
from ark.utils.metacluster_remap_gui import (MetaClusterGui,
                                             colormap_helper,
                                             metaclusterdata_from_files)

## 0: Set root directory and download the example dataset

Here we are using the example data located in `data/example_dataset`. To use your own data, change `base_dir` to point to your own sub-directory within the data folder.

* `base_dir`: the path to all of your imaging data. This directory will also store all of the directories/files created during pixel clustering.

In [ ]:
base_dir = "../data/Revision_tiffs" #set path to folder that "image_data" folder is located

If you would like to test the features in ark with an example dataset, run the cell below. It will download a dataset consisting of 11 FOVs with 22 channels. You may find more information about the example dataset in the [README](../README.md#example-dataset).

If you are using your own data, skip the cell below.

* `overwrite_existing`: If set to `False`, it will not overwrite existing data in `data/example_dataset`. Recommended setting to `False` if you are running Notebooks 1,2,3 and 4 in succession. Set to `True` if you are just running Notebook 2.

In [ ]:
##Skip if using own data

#example_dataset.get_example_dataset(dataset="cluster_pixels", save_dir = base_dir, overwrite_existing = False)

## 1: Set file paths and parameters

* `tiff_dir`: path to the directory containing your imaging data. Images should be single-channel TIFFs.
* `img_sub_folder`: if `tiff_dir` contains an additional subfolder structure, override `None` with the appropriate folder name
* `segmentation_dir`: path to the directory containing your segmentations (which can be generated using `1_Segment_Image_Data.ipynb`). Set this argument to `None` if you do not have segmentation labels or wish to run pixel clustering without them (they are required for cell clustering)
* `seg_suffix`: the suffix plus the file extension of the segmented images for each FOV. Note that these should be the same for all FOVs. This argument will be ignored if `segmentation_dir` is set to `None`
* `pixie_seg_dir`: the created path from the `segmentation_dir`. Is `None` if `segmentation_dir` is `None`.

In [ ]:
tiff_dir = os.path.join(base_dir, "image_data")
img_sub_folder = None
segmentation_dir = None 
seg_suffix = '_whole_cell.tiff'

if segmentation_dir is not None:
    pixie_seg_dir = os.path.join(base_dir, segmentation_dir)
else:
    pixie_seg_dir = None

* `fovs` (optional): set a specific set of fovs to load, default loads all the fovs in `tiff_dir`

In [ ]:
# either get all fovs in the folder...
fovs = io_utils.list_folders(tiff_dir)

# List FOVs, excluding hidden/system files like .DS_Store
#fovs = [f for f in os.listdir(tiff_dir) if not f.startswith('.') and os.path.isdir(os.path.join(tiff_dir, f))]

# ... or optionally, select a specific set of fovs manually
# fovs = ["fov14"]

### Define multiprocessing parameters

Turning on multiprocessing provides a speed boost; however, it is not always cross-platform compatible. If you receive errors such as hanging cells without progress updates, try setting `multiprocess` back to `False`.

In [ ]:
# set to True to turn on multiprocessing
multiprocess = True

# define the number of FOVs to process in parallel, ignored if multiprocessing is set to False
batch_size = 5

## 2: Preprocess

Set a prefix to be applied to all data directories/files created during pixel clustering. If the prefix is not set, a default of the datetime at the start of the run is used.

In [ ]:
# explicitly set pixel_cluster_prefix to override datetime default
pixel_cluster_prefix = "Revision"

if pixel_cluster_prefix is None:
    pixel_cluster_prefix = dt.now().strftime('%Y-%m-%dT%H:%M:%S')

The following data directories/files will be created with names prefixed by `pixel_cluster_prefix`:

* `pixel_output_dir`: directory name where the pixel clustering outputs are stored
* `preprocessed_dir`: directory name where the preprocessed pixel data are stored
* `subsetted_dir`: directory name where the subsetted pixel data are stored
* `norm_vals_name`: file name where the values used to normalize each channel are stored

In [ ]:
# define the output directory using the specified pixel cluster prefix
pixel_output_dir = os.path.join("pixie", "%s_pixel_output_dir" % pixel_cluster_prefix)
if not os.path.exists(os.path.join(base_dir, pixel_output_dir)):
    os.makedirs(os.path.join(base_dir, pixel_output_dir))

# define the preprocessed pixel data folders
pixel_data_dir = os.path.join(pixel_output_dir, 'pixel_mat_data')
pixel_subset_dir = os.path.join(pixel_output_dir, 'pixel_mat_subset')
norm_vals_name = os.path.join(pixel_output_dir, 'channel_norm_post_rownorm.feather')

For certain channels, such as membraneous tumor markers, the subcellular localization of the marker isn't important. Instead, what matters is that cells which are positive for the marker show up as positive. In these cases, we have sometimes found it useful to add additional blurring to these markers before clustering. This ensures that more of the pixels within the cell are positive for the marker, instead of only a few pixels at the border, especially for cells which are under-segmented. However, higher blur levels will also cause more of the pixels in neighboring cells to show up as positive. Therefore, this works best when you have other, robust markers (like CD45) which you can use to determine which cells are false positives for the blurred channel. If you have markers in your panel which fit this description, you can add them in the cell below. Then, when specifying the list of markers to include for clustering, make sure to add `{marker_name}_smoothed`, as that is what the TIFF will be called.

Skip this cell if you don't want to add an additional blur.

In [ ]:
##SKIP

# set an optional list of markers for additional blurring
blurred_channels = ["ECAD"]
smooth_vals = 6

pixel_cluster_utils.smooth_channels(
    fovs=fovs,
    tiff_dir=tiff_dir,
    img_sub_folder=img_sub_folder,
    channels=blurred_channels,
    smooth_vals=smooth_vals,
)

Sometimes, markers will have background staining that you'll want to filter out for the clustering process. Define the name of the marker you want to filter using `filter_channel`. If the marker is only present in the nucleus, set `exclude = False` to filter it out from the membrane. Conversely, if the marker is only present in the membrane, set `exclude = True` to filter it out from the nucleus.

When specifying the list of markers to include for clustering, make sure to add `{marker_name}_nuc_exclude` or `{marker_name}_nuc_include` depending on what type of signal was filtered out.

Skip this cell if you don't want to run marker filtering.

In [ ]:
##SKIP

filter_channel = 'CD163'
nuclear_exclude = True

pixel_cluster_utils.filter_with_nuclear_mask(
    fovs=fovs,
    tiff_dir=tiff_dir,
    seg_dir=os.path.join(base_dir, segmentation_dir),
    channel=filter_channel,
    nuc_seg_suffix="_nuclear.tiff",
    img_sub_folder=img_sub_folder,
    exclude=nuclear_exclude
)

Set the following arguments:

* `channels`: channels to run pixel clustering on
* `blur_factor`: sigma (standard deviation) for the Gaussian blur. Higher values are more aggressive in smoothing signal.
* `subset_proportion`: the fraction of pixels to take from each FOV for training. Sampling is random.

In [ ]:
channels = ["CD45", "Collagen", "Ecadherin", "SMA", "Podoplanin", "VIM", "S100A4", "CD45RO", "CD16", "CD163", "CK", "TFF1", "PDL1", "PD1", "CD31", "Tox_Tox2", "CD57", "FOXP3", "CD4", "NCAD", "CD68", "IL6", "CD20", "CD8", "PDGFRa", "IL8", "GATA6", "CD45RA", "GZMB", "KI67", "CXCL12", "CD3", "S100A2", "FAP_alpha", "CD173", "HLADR", "CD105", "KRT17"]

blur_factor = 2
subset_proportion = 0.1

During pixel preprocessing, the following is done for each FOV:

* Gaussian blur each channel separately
* Remove empty pixels
* For the remaining pixels, normalize each pixel by the sum of all the channels
* Subset a `subset_proportion` fraction of non-empty, normalized pixels. This creates the subsetted dataset for training

Note: if you get integer overflow errors loading in your data, try changing the `dtype` argument to a larger type.

In [ ]:
# run pixel data preprocessing
pixie_preprocessing.create_pixel_matrix(
    fovs,
    channels,
    base_dir,
    tiff_dir,
    pixie_seg_dir,
    img_sub_folder=img_sub_folder,
    seg_suffix=seg_suffix,
    pixel_output_dir=pixel_output_dir,
    data_dir=pixel_data_dir,
    subset_dir=pixel_subset_dir,
    norm_vals_name_post_rownorm=norm_vals_name,
    blur_factor=blur_factor,
    subset_proportion=subset_proportion,
    multiprocess=multiprocess,
    batch_size=batch_size
)

## 3: Pixel clustering

### 3.1: Train pixel SOM

Train the pixel SOM using the subsetted data. Training is done using a self-organizing map (SOM).

The following data directories/files will be created for pixel clustering:

* `pixel_som_weights_name`: file name to store the pixel SOM weights
* `pc_chan_avg_som_cluster_name`: file name to store the average channel expression across all pixel SOM clusters
* `pc_chan_avg_meta_cluster_name`: same as above for pixel meta clusters
* `pixel_meta_cluster_remap_name`: file name to store the SOM cluster to meta cluster manual mappings created using the GUI below

In [ ]:
pixel_som_weights_name = os.path.join(pixel_output_dir, 'pixel_som_weights.feather')
pc_chan_avg_som_cluster_name = os.path.join(pixel_output_dir, 'pixel_channel_avg_som_cluster.csv')
pc_chan_avg_meta_cluster_name = os.path.join(pixel_output_dir, 'pixel_channel_avg_meta_cluster.csv')
pixel_meta_cluster_remap_name = os.path.join(pixel_output_dir, 'pixel_meta_cluster_mapping.csv')

Each channel is normalized by their 99.9% value across the entire dataset before training. These values get saved to `norm_vals_name`.

For a full set of parameters you can customize for `train_pixel_som`, please consult <a href=https://ark-analysis.readthedocs.io/en/latest/_markdown/ark.phenotyping.html#ark.phenotyping.pixel_cluster_utils.train_pixel_som>pixel training docs</a>.

In [ ]:
# create the pixel SOM weights
pixel_pysom = pixel_som_clustering.train_pixel_som(
    fovs,
    channels,
    base_dir,
    subset_dir=pixel_subset_dir,
    norm_vals_name=norm_vals_name,
    som_weights_name=pixel_som_weights_name,
    num_passes=1,
    seed=42
)

### 3.2: Assign pixel SOM clusters

Use the SOM weights learned from `train_pixel_som` to assign pixel clusters to the full preprocessed dataset.

Note that each channel is normalized by the respective value stored in `norm_vals_name` (computed in `train_pixel_som`) prior to cluster assignment.

`generate_som_avg_files` will compute the average channel expression across all pixel SOM clusters, as well as the number of pixels in each pixel SOM cluster (the data in `pc_chan_avg_som_cluster_name`). This is needed for consensus clustering.

In [ ]:
# use pixel SOM weights to assign pixel clusters
pixel_som_clustering.cluster_pixels(
    fovs,
    base_dir,
    pixel_pysom,
    data_dir=pixel_data_dir,
    multiprocess=multiprocess,
    batch_size=batch_size
)

# generate the SOM cluster summary files
pixel_som_clustering.generate_som_avg_files(
    fovs,
    channels,
    base_dir,
    pixel_pysom,
    data_dir=pixel_data_dir,
    pc_chan_avg_som_cluster_name=pc_chan_avg_som_cluster_name,
    require_all_som_clusters=False  # Prevents it from requiring a specific number of clusters
)

In [ ]:
#print(len(pixel_pysom.som_clusters_seen))  # This should be a positive integer

### 3.3: Run pixel consensus clustering

Use consensus hierarchical clustering to cluster pixel SOM clusters into a user-defined number of meta clusters. The consensus clusters are trained on the average channel expression across all pixel SOM clusters (the data stored in `pc_chan_avg_som_cluster_name`). These values are z-scored and capped at the value specified in the `cap` argument prior to consensus clustering. This helps improve meta clustering performance.

After consensus clustering, the following are computed by `generate_meta_avg_files`:

* The average channel expression across all pixel meta clusters, and the number of pixels per meta cluster (the data in `pc_chan_avg_meta_cluster_name`)
* The meta cluster mapping for each pixel SOM cluster in `pc_chan_avg_som_cluster_name` (data is resaved, same data except with an associated meta cluster column)

For a full set of parameters you can customize for `pixel_consensus_cluster`, please consult <a href=https://ark-analysis.readthedocs.io/en/latest/_markdown/ark.phenotyping.html#ark.phenotyping.pixel_cluster_utils.pixel_consensus_cluster>pixel consensus clustering docs</a>

* `max_k`: the number of consensus clusters desired
* `cap`: used to clip z-scored values prior to consensus clustering (in the range `[-cap, cap]`)

In [ ]:
max_k = 20
cap = 3

# run hierarchical clustering using average pixel SOM cluster expression
pixel_cc = pixel_meta_clustering.pixel_consensus_cluster(
    fovs,
    channels,
    base_dir,
    max_k=max_k,
    cap=cap,
    data_dir=pixel_data_dir,
    pc_chan_avg_som_cluster_name=pc_chan_avg_som_cluster_name,
    multiprocess=multiprocess,
    batch_size=batch_size
)

# generate the meta cluster summary files
pixel_meta_clustering.generate_meta_avg_files(
    fovs,
    channels,
    base_dir,
    pixel_cc,
    data_dir=pixel_data_dir,
    pc_chan_avg_som_cluster_name=pc_chan_avg_som_cluster_name,
    pc_chan_avg_meta_cluster_name=pc_chan_avg_meta_cluster_name
)

## 4: Visualize results

### 4.1: Interactive adjustments to relabel pixel meta clusters

The visualization shows the z-scored average channel expression per pixel SOM and meta cluster. The heatmaps are faceted by pixel SOM clusters on the left and pixel meta clusters on the right.

## Usage

### Quickstart
- **Select**: Left Click
- **Remap**: **New metacluster** button or Right Click
- **Edit Metacluster Name**: Textbox at bottom right of the heatmaps.

### Selection and remapping details
- To select a SOM cluster, click on its respective position in the **selected** bar. Click on it again to deselect.
- To select a meta cluster, click on its corresponding color in the **metacluster** bar. Click on it again to deselect.
- To remap the selected clusters, click the **New metacluster** button (alternatively, right click anywhere). Note that remapping an entire metacluster deletes it.
- To clear the selected SOM/meta clusters, use the **Clear Selection** button.
- **After remapping a meta cluster, make sure to deselect the newly created one to prevent unwanted combinations.**

### Other features and notes
- You will likely need to zoom out to see the entire visualization. To toggle Zoom, use Ctrl -/Ctrl + on Windows or ⌘ +/⌘ - on Mac.
- The bars at the top show the number of pixels in each SOM cluster.
- The text box at the bottom right allows you to rename a particular meta cluster. This can be useful as remapping may cause inconsistent numbering. **You cannot use the same name for different meta clusters; doing so will cause the next step to fail.**
- Adjust the z-score limit using the slider on the bottom left to adjust your dynamic range.
- When meta clusters are combined or a meta cluster is renamed, the change is immediately saved to `pixel_meta_cluster_remap_name`.
- You won't be able to advance in the notebook until you've clicked `New metacluster` or renamed a meta cluster at least once. If you don't want to make changes, just click `New metacluster` to trigger a save before continuing.

In [ ]:
%matplotlib widget
rc_file_defaults()
plt.ion()

pixel_mcd = metaclusterdata_from_files(
    os.path.join(base_dir, pc_chan_avg_som_cluster_name),
    cluster_type='pixel'
)
pixel_mcd.output_mapping_filename = os.path.join(base_dir, pixel_meta_cluster_remap_name)
pixel_mcg = MetaClusterGui(pixel_mcd, width=17)

In [ ]:
# Save the figure
save_path = "/Users/nicolegross/ark-analysis/data/Revision_tiffs/pixie/Revision_pixel_output_dir/heatmap.pdf"
plt.savefig(save_path, dpi=300)

# Show the plot
plt.show()

Relabel the pixel meta clusters using the mapping, and recompute the meta cluster average files with the new meta cluster names.

In [ ]:
# rename the meta cluster values in the pixel dataset
pixel_meta_clustering.apply_pixel_meta_cluster_remapping(
    fovs,
    channels,
    base_dir,
    pixel_data_dir,
    pixel_meta_cluster_remap_name,
    multiprocess=multiprocess,
    batch_size=batch_size
)

# recompute the mean channel expression per meta cluster and apply these new names to the SOM cluster average data
pixel_meta_clustering.generate_remap_avg_files(
    fovs,
    channels,
    base_dir,
    pixel_data_dir,
    pixel_meta_cluster_remap_name,
    pc_chan_avg_som_cluster_name,
    pc_chan_avg_meta_cluster_name
)

Generate the color scheme returned by the interactive reclustering process. This will be for visualizing the pixel phenotype maps.

In [ ]:
raw_cmap, _ = colormap_helper.generate_meta_cluster_colormap_dict(
    pixel_mcd.output_mapping_filename,
    pixel_mcg.im_cl.cmap
)

### 4.2: Generate pixel phenotype maps

Generate pixel phenotype maps, in which each pixel in the image corresponds to its pixel meta cluster. Select a small subset of your FOVs to view within this notebook. Or if you wish to generate and save a significant amount of FOVs, the masks will be created and saved in batches.

Files will be written as `{fov_name}_pixel_mask.tiff` in `pixel_output_dir`

In [ ]:
# select fovs to display
subset_pixel_fovs = ['JHH317_ROI001', 'JHH317_ROI002', 'JHH317_ROI003','JHH317_ROI004'] #name of folders to display data from

In [ ]:
print(fovs)

In [ ]:
# define the path to the channel file
if img_sub_folder is None:
    chan_file = os.path.join(
        io_utils.list_files(os.path.join(tiff_dir, fovs[0]), substrs=['.tiff'])[0] #ENSURE THIS MATCHES YOUR FILE EXTENSION (TIF VS TIFF)
    )
else:
    chan_file = os.path.join(
        img_sub_folder, io_utils.list_files(os.path.join(tiff_dir, fovs[0], img_sub_folder), substrs=['.tiff'])[0]
    )

# generate and save the pixel cluster masks for each fov in subset_pixel_fovs
data_utils.generate_and_save_pixel_cluster_masks(
    fovs=fovs, #i edited this
    base_dir=base_dir,
    save_dir=os.path.join(base_dir, pixel_output_dir),
    tiff_dir=tiff_dir,
    chan_file=chan_file,
    pixel_data_dir=pixel_data_dir,
    cluster_id_to_name_path=os.path.join(base_dir, pixel_meta_cluster_remap_name),
    pixel_cluster_col='pixel_meta_cluster',
    sub_dir='pixel_masks',
    name_suffix='_pixel_mask',
)

In [ ]:
#print("FOVs:", fovs)

Save the colored pixel masks for each FOV in `subset_pixel_fovs`.

In [ ]:
plot_utils.save_colored_masks(
    fovs=fovs, #i edited this
    mask_dir=os.path.join(base_dir, pixel_output_dir, "pixel_masks"),
    save_dir=os.path.join(base_dir, pixel_output_dir, "pixel_mask_colored"),
    cluster_id_to_name_path=os.path.join(base_dir, pixel_meta_cluster_remap_name),
    metacluster_colors=raw_cmap,
    cluster_type="pixel"
)

Load a subset of the pixel cluster masks that you would like to preview. If the dimensions or text size of the plot need to be adjusted for optimal viewing, change the `figsize` and `dpi` parameters.

In [ ]:
#i added this

subset_pixel_fovs = [f for f in os.listdir(os.path.join(base_dir, pixel_output_dir, "pixel_masks"))
                     if f.endswith("_pixel_mask.tiff")]
subset_pixel_fovs = [f.replace("_pixel_mask.tiff", "") for f in subset_pixel_fovs]

In [ ]:
for pixel_fov in subset_pixel_fovs:
    pixel_cluster_mask = load_utils.load_imgs_from_dir(
        data_dir=os.path.join(base_dir, pixel_output_dir, "pixel_masks"),
        files=[pixel_fov + "_pixel_mask.tiff"],
        trim_suffix="_pixel_mask",
        match_substring="_pixel_mask",
        xr_dim_name="pixel_mask",
        xr_channel_names=None,
    )

    plot_utils.plot_pixel_cell_cluster(
        pixel_cluster_mask,
        [pixel_fov],
        os.path.join(base_dir, pixel_meta_cluster_remap_name),
        metacluster_colors=raw_cmap, 
        figsize=(5,5), 
        dpi=150
    )

## 5: Save parameters for use in cell clustering

The following parameters are saved:

* `fovs`: fovs in `pixel_data_dir`
* `channels`: channels used for clustering
* `segmentation_dir`: path to the directory containing your segmentated images for each FOV (can be generated using `1_Segment_Image_Data.ipynb`)
* `seg_suffix`: suffix plus the file extension of the segmented images for each FOV
* `pixel_data_dir`: name of the directory containing the full pixel data with the pixel SOM and meta cluster assignments
* `pc_chan_avg_som_cluster_name`: name of the file containing the average channel expression per pixel SOM cluster
* `pc_chan_avg_meta_cluster_name`: same as above for pixel meta clusters

The file will be saved to `{pixel_cluster_prefix}_cell_clustering_params.json` and will be placed in `pixel_output_dir`. Note that the `pixel_output_dir` you use in `2_Pixie_Cluster_Pixels.ipynb` should be the same as in `3_Pixie_Cluster_Cells.ipynb`.

In [ ]:
# define the params dict
cell_clustering_params = {
    'fovs': io_utils.remove_file_extensions(io_utils.list_files(os.path.join(base_dir, pixel_data_dir), substrs='.feather')),
    'channels': channels,
    'tiff_dir': tiff_dir,
    'img_sub_folder': img_sub_folder,
    'segmentation_dir': segmentation_dir,
    'seg_suffix': seg_suffix,
    'pixel_data_dir': pixel_data_dir,
    'pc_chan_avg_som_cluster_name': pc_chan_avg_som_cluster_name,
    'pc_chan_avg_meta_cluster_name': pc_chan_avg_meta_cluster_name
}

# save the params dict
with open(os.path.join(base_dir, pixel_output_dir, 'cell_clustering_params.json'), 'w') as fh:
    json.dump(cell_clustering_params, fh)

### 5.1 Save images for Mantis Viewer

Mantis Viewer is a visualization tool for multi-dimensional imaging in pathology. Learn more about Mantis Viewer in the [README](../README.md#mantis-viewer).

In [ ]:
plot_utils.create_mantis_dir(
    fovs=subset_pixel_fovs,
    mantis_project_path=os.path.join(base_dir, "mantis"),
    img_data_path=tiff_dir,
    mask_output_dir=os.path.join(base_dir, pixel_output_dir, "pixel_masks"),
    mapping = os.path.join(base_dir, pixel_meta_cluster_remap_name),
    seg_dir=pixie_seg_dir,
    mask_suffix="_pixel_mask",
    seg_suffix_name=seg_suffix,
    img_sub_folder=img_sub_folder
)

In [ ]:
#### extract info to create a csv which contains, for each fov/file: pixel count for each metacluster, total area of each metacluster, and x & y coords

import os
import pandas as pd
import numpy as np

# Path to the pixel_mat_data directory
pixel_output_dir = os.path.join(base_dir, "pixie", "Revision_pixel_output_dir")
pixel_mat_data_dir = os.path.join(pixel_output_dir, "pixel_mat_data")



In [ ]:
# Function to load all feather files in the directory
def load_all_feather_files(directory):
    # List all files in the directory
    feather_files = [f for f in os.listdir(directory) if f.endswith('.feather')]
    
    # Load all Feather files into a single DataFrame
    all_data = []
    for feather_file in feather_files:
        file_path = os.path.join(directory, feather_file)
        df = pd.read_feather(file_path)
        all_data.append(df)
    
    # Concatenate all DataFrames into one
    combined_data = pd.concat(all_data, ignore_index=True)
    return combined_data

In [ ]:
# Function to load all feather files in the directory
def load_all_feather_files(directory):
    # List all files in the directory
    feather_files = [f for f in os.listdir(directory) if f.endswith('.feather')]
    
    # Load all Feather files into a single DataFrame
    all_data = []
    for feather_file in feather_files:
        file_path = os.path.join(directory, feather_file)
        df = pd.read_feather(file_path)
        all_data.append(df)
    
    # Concatenate all DataFrames into one
    combined_data = pd.concat(all_data, ignore_index=True)
    return combined_data

# Load all the data from the Feather files
pixel_data = load_all_feather_files(pixel_mat_data_dir)


In [ ]:
# Define a function to calculate pixel area and centroid
def calculate_cluster_stats(data):
    # List to store output for each FOV
    output = []

    # Group by FOV and Meta-cluster
    fovs = data['fov'].unique()
    for fov in fovs:
        fov_data = data[data['fov'] == fov]

        # Group by Meta-cluster and calculate statistics
        meta_clusters = fov_data['pixel_meta_cluster_rename'].unique()
        for cluster in meta_clusters:
            cluster_data = fov_data[fov_data['pixel_meta_cluster_rename'] == cluster]
            
            # Pixel count
            pixel_count = len(cluster_data)
            
            # Total area (5x5 um² per pixel)
            #total_area = pixel_count * 25  # 25 um² per pixel
            
            # Calculate centroid (mean row and column index)
            centroid_x = np.mean(cluster_data['column_index'])
            centroid_y = np.mean(cluster_data['row_index'])

            # Append the results
            output.append({
                'file_name': fov,
                'cluster': cluster,
                'pixel_count': pixel_count,
                #'total_area_umsq': total_area,
                'centroid_x': centroid_x,
                'centroid_y': centroid_y
            })
    
    # Convert the output list into a DataFrame
    output_df = pd.DataFrame(output)
    
    return output_df


In [ ]:
# Calculate the cluster statistics
cluster_stats = calculate_cluster_stats(pixel_data)

# Save the output to CSV inside the pixel_output_dir
csv_save_path = os.path.join(pixel_output_dir, "pixel_meta_cluster_summary.csv")
cluster_stats.to_csv(csv_save_path, index=False)

print(f"Summary file generated: {csv_save_path}")